<a href="https://colab.research.google.com/github/NustinJewman/PythonFFB/blob/main/data_munging_notes_pt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/74b84c5fb2371b954b52b4f67ae5220930d57861/fantasypros/fp_projections.csv')

In [ ]:
rb_df = df.loc[df['Pos'] == 'WR']
rb_df.head()
df.sort_values(by='ReceivingYds', ascending=False).head(10)